Let us agggregate data in 5-day intervals. That way we can use the variables whose temporal resolution are of 5-day granularity and also shortens the length of time series. It is likely(?) that we do not need track every thing on a daily basis.

- Dec. 29, 2025
- HN

In [1]:
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
import pandas as pd
import numpy as np
import random
import os, os.path, pickle, sys

import matplotlib
import matplotlib.pyplot as plt

from typing import List, Optional, Any

import configparser
import argparse

# from pathlib import Path
# PROJECT_ROOT = Path.cwd().parent
# sys.path.insert(0, str(PROJECT_ROOT))
# from transformersCores import config
# from transformersCores import config as cfg

import transformersCores.config as cfg
import transformersCores.preprocess as prp

In [2]:
import importlib
importlib.reload(cfg);
importlib.reload(prp);

In [3]:
dir_base = "./../../../01_research_data/Ehsan/wheat/"
data_base = dir_base + "Data/"
raw_dir = os.path.join(data_base, "00_raw_by_location/")
configs_dir = dir_base + "configs/"

In [4]:
params = cfg.aggregateParameters()
params.readFromFile(configs_dir + "Scenario_1_aggr_cols.cfg")
# params = vars(params)

In [5]:
# params_v2 = cfg.aggregateParameters()
# params_v2.load_config_from_file(configs_dir + "Scenario_1_aggr_cols.cfg")

### Read Data

In [6]:
filename = raw_dir + "raw_all_locations_separate_daily_monthly_fiveDay.sav"
data_dict = pd.read_pickle(filename)

In [7]:
data_dict.keys()

dict_keys(['wheat_all_locs_raw_daily', 'wheat_all_locs_raw_monthly', 'wheat_all_locs_raw_fiveDay', 'varieties_traits', 'source_code', 'Author', 'Date', 'Note'])

In [8]:
raw_daily = data_dict["wheat_all_locs_raw_daily"]
raw_fiveDay = data_dict["wheat_all_locs_raw_fiveDay"]
varieties_traits = data_dict["varieties_traits"]

In [9]:
raw_daily.head(2)

,location,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,...,diurnal_temp_range_c,precip_dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,final_season,freezing_dd_cday,heat_dd_cday,year,planting_date,harvest_date
0,farmington,2001-10-01,274,0,Emergence,0.0,25.25,7.35,1.41,4.0,...,17.9,0.0,0.0,0,Preseason,0.0,2.46,2001,NaN,NaN
1,farmington,2001-10-02,275,1,Emergence,0.0,21.65,8.05,1.18,3.2,...,13.6,0.0,0.0,0,Preseason,0.0,0.00,2001,NaN,NaN


In [10]:
params = cfg.aggregateParameters()
params.readFromFile(configs_dir + "Scenario_1_aggr_cols.cfg")

daily_converted_2_5day = prp.aggregate_per5day(params=params, daily_df=raw_daily, fiveDay_df=raw_fiveDay)

In [11]:
raw_daily[(raw_daily["location"] == "walla_walla") & (raw_daily["year"] == 2003)].head(5)

,location,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,...,diurnal_temp_range_c,precip_dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,final_season,freezing_dd_cday,heat_dd_cday,year,planting_date,harvest_date
116731,walla_walla,2003-01-01,1,0,Emergence,2.2,7.35,0.55,0.19,0.9,...,6.8,0.323529,0.0,1,Preseason,3.68,0.0,2003,2003-03-24,2003-07-25
116732,walla_walla,2003-01-02,2,1,Emergence,6.1,11.75,5.45,0.29,1.6,...,6.3,0.968254,0.0,1,Preseason,0.00,0.0,2003,2003-03-24,2003-07-25
116733,walla_walla,2003-01-03,3,2,Emergence,2.4,10.95,6.85,0.31,1.3,...,4.1,0.585366,0.0,1,Preseason,0.00,0.0,2003,2003-03-24,2003-07-25
116734,walla_walla,2003-01-04,4,3,Emergence,3.7,10.55,7.15,0.27,1.4,...,3.4,1.088235,0.0,1,Preseason,0.00,0.0,2003,2003-03-24,2003-07-25
116735,walla_walla,2003-01-05,5,4,Emergence,0.0,8.35,2.75,0.24,0.9,...,5.6,0.000000,0.0,0,Preseason,1.48,0.0,2003,2003-03-24,2003-07-25


In [12]:
raw_fiveDay[(raw_fiveDay["date"].dt.year == 2003) & (raw_fiveDay["location"] == "walla_walla")].head(5)

,date,doy,location,std_precip_evap_unitless,palmer_drought_unitless
23417,2003-01-05,5,walla_walla,0.90,-3.53
23418,2003-01-10,10,walla_walla,0.90,-3.45
23419,2003-01-15,15,walla_walla,-0.17,-3.40
23420,2003-01-20,20,walla_walla,-0.80,-3.37
23421,2003-01-25,25,walla_walla,0.03,-3.30


In [13]:
end_date = pd.to_datetime("2003-01-05")
loc = "walla_walla"
# end_date = raw_fiveDay["date"][0]
five_days = pd.date_range(end=end_date, periods=5, freq='D')
a_col = "precip_mmday"

raw_daily[(raw_daily["location"] == loc) & (raw_daily["date"].isin(five_days))][a_col].sum()

np.float64(14.400000000000002)

In [14]:
a_col = "sum_precip_mmday"
daily_converted_2_5day[(daily_converted_2_5day["location"] == loc) & \
                       (daily_converted_2_5day["date"] == end_date)][a_col]

24542    14.4
Name: sum_precip_mmday, dtype: float64

In [15]:
daily_converted_2_5day.head(2)

,location,date,mean_avg_rh_perc,mean_avg_temp_c,mean_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,mean_heat_dd_cday,...,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms,final_season,season,stage,planting_date,harvest_date
0,almira,2001-10-02,43.25,15.20,0.0,1.20,0.5,14.00,0.000,0.68,...,0.004335,163.45,1.245,0.0,2.70,Preseason,Preseason,Emergence,NaN,NaN
1,almira,2001-10-07,41.92,9.79,0.0,2.13,4.0,12.96,1.344,0.00,...,0.003190,148.38,0.844,0.0,3.26,Preseason,Preseason,Emergence,NaN,NaN


In [16]:
raw_daily[(raw_daily["location"] == "walla_walla") & (raw_daily["year"] == 2001)]["planting_date"].unique()

array([nan], dtype=object)

In [17]:
daily_converted_2_5day['date'] = pd.to_datetime(daily_converted_2_5day['date'])
daily_converted_2_5day['planting_date'] = pd.to_datetime(daily_converted_2_5day['planting_date'])
daily_converted_2_5day['harvest_date'] = pd.to_datetime(daily_converted_2_5day['harvest_date'])

In [18]:
daily_converted_2_5day_backup = daily_converted_2_5day.copy()

In [19]:
daily_converted_2_5day = daily_converted_2_5day_backup.copy()

In [20]:
# daily_converted_2_5day.dropna(subset=['planting_date'], inplace=True)
# daily_converted_2_5day.sort_values(["location", "date"], inplace=True)
# daily_converted_2_5day.reset_index(drop=True, inplace=True)

# daily_converted_2_5day.head(3)

In [21]:
daily_converted_2_5day_backup.head(3)

,location,date,mean_avg_rh_perc,mean_avg_temp_c,mean_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,mean_heat_dd_cday,...,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms,final_season,season,stage,planting_date,harvest_date
0,almira,2001-10-02,43.25,15.20,0.0,1.20,0.5,14.00,0.000,0.68,...,0.004335,163.45,1.245,0.0,2.70,Preseason,Preseason,Emergence,NaT,NaT
1,almira,2001-10-07,41.92,9.79,0.0,2.13,4.0,12.96,1.344,0.00,...,0.003190,148.38,0.844,0.0,3.26,Preseason,Preseason,Emergence,NaT,NaT
2,almira,2001-10-12,63.49,6.67,0.0,1.62,9.0,9.64,2.380,0.00,...,0.004350,99.80,0.380,0.4,4.90,Preseason,Preseason,Emergence,NaT,NaT


In [22]:
super_df_oct = prp.TS_df_givenMonth_prior2Plant(daily_converted_2_5day_backup, start_month=10)

## Check point

In [23]:
super_df_oct.head(3)

,location,planting_date,harvest_date,mean_avg_rh_perc,mean_avg_temp_c,mean_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,...,mean_min_rh_perc,mean_min_temp_c,mean_potential_evapo_mmday,mean_precip_dtr_mmdayc,sum_precip_mmday,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms
0,almira,2002-04-08,2002-08-12,date 2001-10-02 43.25 2001-10-07 41.92 2...,date 2001-10-02 15.20 2001-10-07 9.79 2...,date 2001-10-02 0.00 2001-10-07 0....,date 2001-10-02 1.20 2001-10-07 2.13 200...,date 2001-10-02 0.5 2001-10-07 4.0 2...,date 2001-10-02 14.00 2001-10-07 12.96 2...,date 2001-10-02 0.000 2001-10-07 1.344 2...,...,date 2001-10-02 23.70 2001-10-07 21.52 2...,date 2001-10-02 8.20 2001-10-07 3.31 2...,date 2001-10-02 3.25 2001-10-07 2.78 200...,date 2001-10-02 0.000000 2001-10-07 0.00...,date 2001-10-02 0.0 2001-10-07 0.0 2001-...,date 2001-10-02 0.004335 2001-10-07 0.00...,date 2001-10-02 163.45 2001-10-07 148.38...,date 2001-10-02 1.245 2001-10-07 0.844 2...,date 2001-10-02 0.0 2001-10-07 0.0 2001-...,date 2001-10-02 2.70 2001-10-07 3.26 200...
1,almira,2003-04-29,2003-08-20,date 2002-10-02 48.15 2002-10-07 66.29 2...,date 2002-10-02 16.058333 2002-10-07 11....,date 2002-10-02 1929.641667 2002-10-07 1...,date 2002-10-02 3.391667 2002-10-07 1.38...,date 2002-10-02 238.833333 2002-10-07 27...,date 2002-10-02 15.95 2002-10-07 12.92 2...,date 2002-10-02 6.366667 2002-10-07 9....,...,date 2002-10-02 23.416667 2002-10-07 41....,date 2002-10-02 8.083333 2002-10-07 5....,date 2002-10-02 4.966667 2002-10-07 2.22...,date 2002-10-02 0.000000 2002-10-07 0.05...,date 2002-10-02 0.0 2002-10-07 2.1 2002-...,date 2002-10-02 0.005242 2002-10-07 0.00...,date 2002-10-02 239.183333 2002-10-07 13...,date 2002-10-02 1.393333 2002-10-07 0.60...,date 2002-10-02 0.0 2002-10-07 0.4 2002-...,date 2002-10-02 3.45 2002-10-07 3.02 200...
2,almira,2004-04-01,2004-08-12,date 2003-10-02 36.9625 2003-10-07 36.67...,date 2003-10-02 19.3875 2003-10-07 17.18...,date 2003-10-02 2103.35 2003-10-07 2108....,date 2003-10-02 1.625 2003-10-07 1.210 2...,date 2003-10-02 252.0 2003-10-07 277.0 2...,date 2003-10-02 15.925 2003-10-07 12.500...,date 2003-10-02 3.125 2003-10-07 3.520...,...,date 2003-10-02 19.675 2003-10-07 20.180...,date 2003-10-02 11.425 2003-10-07 10.930...,date 2003-10-02 4.975 2003-10-07 3.260 2...,date 2003-10-02 0.000000 2003-10-07 0.00...,date 2003-10-02 0.0 2003-10-07 0.0 200...,date 2003-10-02 0.004848 2003-10-07 0.00...,date 2003-10-02 220.175 2003-10-07 143.3...,date 2003-10-02 1.7925 2003-10-07 1.4620...,date 2003-10-02 0.0 2003-10-07 0.0 2003-...,date 2003-10-02 3.10 2003-10-07 3.00 200...


In [24]:
t2 = super_df_oct.loc[0, "harvest_date"]
t1 = super_df_oct.loc[0, "planting_date"]
t1 = pd.Timestamp(f"{t1.year - 1}-10-01")

days_between = (t2 - t1).days
days_between

315

In [25]:
daily_converted_2_5day[(daily_converted_2_5day["location"] == "almira") & \
                       (daily_converted_2_5day['date'].dt.year == 2002)].head(3)

,location,date,mean_avg_rh_perc,mean_avg_temp_c,mean_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,mean_heat_dd_cday,...,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms,final_season,season,stage,planting_date,harvest_date
19,almira,2002-01-05,88.62,-1.67,0.0,0.00,2.0,6.76,9.28,0.0,...,0.003320,45.36,0.056,0.4,2.1,Preseason,Preseason,Emergence,2002-04-08,2002-08-12
20,almira,2002-01-10,86.42,1.32,0.0,1.02,7.0,5.86,5.84,0.0,...,0.004014,37.54,0.084,0.4,2.9,Preseason,Preseason,Emergence,2002-04-08,2002-08-12
21,almira,2002-01-15,82.62,-1.38,0.0,0.92,12.0,7.98,9.60,0.0,...,0.003086,50.80,0.116,0.2,3.0,Preseason,Preseason,Emergence,2002-04-08,2002-08-12


In [26]:
locations_list = daily_converted_2_5day["location"].unique()
loc_ = locations_list[0]

loc_yr = daily_converted_2_5day.loc[daily_converted_2_5day["location"] == \
                                      loc_, "harvest_date"].unique()[-1]
loc_yr

Timestamp('2023-08-21 00:00:00')

In [27]:
test_super = super_df_oct[(super_df_oct["location"] == loc_) & (super_df_oct["harvest_date"] == loc_yr)]
test_super

,location,planting_date,harvest_date,mean_avg_rh_perc,mean_avg_temp_c,mean_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,...,mean_min_rh_perc,mean_min_temp_c,mean_potential_evapo_mmday,mean_precip_dtr_mmdayc,sum_precip_mmday,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms
20,almira,2023-04-26,2023-08-21,date 2022-10-02 55.85 2022-10-07 45.60 2...,date 2022-10-02 17.95 2022-10-07 18.64 2...,date 2022-10-02 2141.50 2022-10-07 2141....,date 2022-10-02 1.15 2022-10-07 0.55 202...,date 2022-10-02 273.5 2022-10-07 277.0 2...,date 2022-10-02 14.60 2022-10-07 13.70 2...,date 2022-10-02 3.80 2022-10-07 2.66 202...,...,date 2022-10-02 27.45 2022-10-07 22.30 2...,date 2022-10-02 10.65 2022-10-07 11.79 2...,date 2022-10-02 3.05 2022-10-07 3.00 202...,date 2022-10-02 0.000000 2022-10-07 0.00...,date 2022-10-02 0.0 2022-10-07 0.0 202...,date 2022-10-02 0.007070 2022-10-07 0.00...,date 2022-10-02 165.85 2022-10-07 158.18...,date 2022-10-02 1.190 2022-10-07 1.400 2...,date 2022-10-02 0.0 2022-10-07 0.0 2022-...,date 2022-10-02 2.40 2022-10-07 2.24 202...


In [28]:
start_date = pd.Timestamp(year=loc_yr.year - 1, month=10, day=1)

test_df = daily_converted_2_5day[(daily_converted_2_5day["location"] == loc_) & \
                                 (daily_converted_2_5day["date"] >= start_date) & \
                                 (daily_converted_2_5day["date"] <= loc_yr)]

In [29]:
a_col = "mean_dailyGDD_diff_cday"
sum(test_df[a_col].values - test_super[a_col].item().values)

np.float64(0.0)

### Time series 3 months prior planting date as opposed to prior Oct.

In [30]:
import importlib
importlib.reload(cfg);
importlib.reload(prp);

In [31]:
super_df_3months = prp.TS_df_countMonths_prior2Plant(daily_converted_2_5day_backup, months_prior=3)

## Check point

In [32]:
locations_list = daily_converted_2_5day["location"].unique()
loc_ = locations_list[0]

loc_yr = daily_converted_2_5day.loc[daily_converted_2_5day["location"] == \
                                      loc_, "planting_date"].unique()[2]
loc_yr

Timestamp('2003-04-29 00:00:00')

In [33]:
test_super = super_df_3months[(super_df_3months["location"] == loc_) & \
                              (super_df_3months["planting_date"] == loc_yr)]
test_super

,location,planting_date,harvest_date,mean_avg_rh_perc,mean_avg_temp_c,mean_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,...,mean_min_rh_perc,mean_min_temp_c,mean_potential_evapo_mmday,mean_precip_dtr_mmdayc,sum_precip_mmday,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms
1,almira,2003-04-29,2003-08-20,date 2003-01-05 87.44 2003-01-10 81.31 2...,date 2003-01-05 2.30 2003-01-10 -1.93 2...,date 2003-01-05 0.00 2003-01-10 0....,date 2003-01-05 9.800000e-01 2003-01-10 ...,date 2003-01-05 2.0 2003-01-10 7.0 2...,date 2003-01-05 4.38 2003-01-10 7.08 2...,date 2003-01-05 4.120 2003-01-10 9.700...,...,date 2003-01-05 74.88 2003-01-10 65.96 2...,date 2003-01-05 0.11 2003-01-10 -5.47 2...,date 2003-01-05 0.30 2003-01-10 0.32 200...,date 2003-01-05 1.371110 2003-01-10 0.00...,date 2003-01-05 24.2 2003-01-10 0.0 200...,date 2003-01-05 0.004370 2003-01-10 0.00...,date 2003-01-05 40.90 2003-01-10 61.34...,date 2003-01-05 0.078 2003-01-10 0.114 2...,date 2003-01-05 0.8 2003-01-10 0.0 2003-...,date 2003-01-05 3.26 2003-01-10 2.56 200...


In [34]:
three_months_prior = loc_yr - pd.DateOffset(months=3)
start_date = three_months_prior.replace(day=1)

harvest_date = daily_converted_2_5day.loc[(daily_converted_2_5day["location"] == loc_) & \
                           (daily_converted_2_5day["planting_date"] == loc_yr)]["harvest_date"].unique()[0]

test_df = daily_converted_2_5day[(daily_converted_2_5day["location"] == loc_) & \
                                 (daily_converted_2_5day["date"] >= start_date) & \
                                 (daily_converted_2_5day["date"] <= harvest_date)]
test_df.head(3)

,location,date,mean_avg_rh_perc,mean_avg_temp_c,mean_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,mean_heat_dd_cday,...,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms,final_season,season,stage,planting_date,harvest_date
82,almira,2003-01-05,87.44,2.30,0.0,0.98,2.0,4.38,4.12,0.0,...,0.004370,40.90,0.078,0.8,3.26,Preseason,Preseason,Emergence,2003-04-29,2003-08-20
83,almira,2003-01-10,81.31,-1.93,0.0,0.48,7.0,7.08,9.70,0.0,...,0.002918,61.34,0.114,0.0,2.56,Preseason,Preseason,Emergence,2003-04-29,2003-08-20
84,almira,2003-01-15,86.49,-0.04,0.0,0.52,12.0,4.34,6.44,0.0,...,0.003696,45.84,0.062,0.4,1.76,Preseason,Preseason,Emergence,2003-04-29,2003-08-20


In [35]:
a_col = "mean_dap"
sum(test_df[a_col].values - test_super[a_col].item().values)

np.float64(0.0)